In [1]:
!pip install -q requests openseries

In [2]:
from requests import get as requests_get
import plotly.io as pio
from sys import modules as sys_modules

from openseries import (
    OpenTimeSeries,
    OpenFrame,
    ReturnSimulation,
    report_html,
    ValueType,
    efficient_frontier,
    prepare_plot_data,
    sharpeplot,
    load_plotly_dict,
    get_previous_business_day_before_today,
)

figdict, _ = load_plotly_dict()
seed = 55

In [3]:
if "google.colab" in sys_modules:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"

In [4]:
def make_fund_basket(positions: dict[str, float], timeout: int = 10) -> OpenFrame:
    response = requests_get(url="https://api.captor.se/public/api/nav", timeout=timeout)
    response.raise_for_status()

    found, weights, series = set(), [], []
    result = response.json()
    for data in result:
        if data["isin"] in positions:
            found.add(data["isin"])
            weights.append(positions[data["isin"]])
            series.append(
                OpenTimeSeries.from_arrays(
                    name=data["longName"],
                    isin=data["isin"],
                    baseccy=data["currency"],
                    dates=data["dates"],
                    values=data["navPerUnit"],
                    valuetype=ValueType.PRICE,
                )
            )

    if len(set(positions.keys()) - found) != 0:
        raise ValueError(f"Request for NAV series failed. Missing ISINs are: {set(positions.keys()) - found}")

    return OpenFrame(constituents=series, weights=weights)

In [5]:
funds = {
    "SE0015243886": 0.2,
    "SE0011337195": 0.2,
    "SE0011670843": 0.2,
    "SE0017832280": 0.2,
    "SE0017832330": 0.2,
}
basket = make_fund_basket(positions=funds)
basket = basket.value_nan_handle().trunc_frame().to_cumret()

In [6]:
portfolio = OpenTimeSeries.from_df(dframe=basket.make_portfolio(name="Portfolio"))
basket = basket.add_timeseries(new_series=portfolio)

In [7]:
figure, _ = basket.plot_series(tick_fmt=".1%", auto_open=False, output_type="div", add_logo=False)
figure = figure.update_layout(
    height=600,
    font_size=10,
    legend=dict(
        yanchor="bottom",
        y=-0.3,
        xanchor="right",
        x=0.98,
        orientation="h"
    )
)
figure = figure.update_layout(font_size=14, width=1100, height=600)

In [8]:
figure.show(config=figdict["config"])

In [9]:
dataframe = basket.all_properties(properties=[
        "arithmetic_ret",
        "vol",
        "ret_vol_ratio",
        "sortino_ratio",
        "worst_month",
        "cvar_down",
        "first_indices",
        "last_indices",
    ]
)

In [10]:
dataframe.columns = dataframe.columns.droplevel(level=1)

formats = [
        "{:.2%}",
        "{:.2%}",
        "{:.2f}",
        "{:.2f}",
        "{:.2%}",
        "{:.2%}",
        "{:%Y-%m-%d}",
        "{:%Y-%m-%d}",
    ]

for item, f in zip(dataframe.index, formats):
    dataframe.loc[item] = dataframe.loc[item].apply(
        lambda x, fmt=f: x if isinstance(x, str) else fmt.format(x)
    )

In [11]:
dataframe

,Captor Scilla Global Equity,Captor Aster Global Credit Short-Term,Captor Aster Global High Yield,Captor Aster Global Credit,Captor Dahlia Green Bond,Portfolio
Arithmetic return,10.68%,5.26%,11.13%,6.77%,4.67%,7.70%
Volatility,10.33%,1.63%,6.62%,8.23%,2.90%,4.21%
Return vol ratio,1.03,3.23,1.68,0.82,1.61,1.83
Sortino ratio,1.44,4.91,2.54,1.22,2.58,2.69
Worst month,-7.55%,-0.25%,-2.70%,-5.49%,-2.20%,-2.91%
CVaR 95.0%,-1.59%,-0.25%,-0.97%,-1.11%,-0.38%,-0.63%
first indices,2022-12-06,2022-12-06,2022-12-06,2022-12-06,2022-12-06,2022-12-06
last indices,2025-06-23,2025-06-23,2025-06-23,2025-06-23,2025-06-23,2025-06-23


In [12]:
hy = OpenTimeSeries.from_df(dframe=basket.tsdf.loc[:, ("Captor Aster Global High Yield", ValueType.PRICE)])
eq = OpenTimeSeries.from_df(dframe=basket.tsdf.loc[:, ("Captor Scilla Global Equity", ValueType.PRICE)])
compare = OpenFrame(constituents=[hy, eq])
report, _ = report_html(
    data=compare,
    bar_freq="BQE",
    vertical_legend=False,
    output_type="div",
)
report = report.update_layout(font_size=10, width=1100, height=650)

In [17]:
report.show(config=figdict["config"])

In [14]:
series = ReturnSimulation.from_merton_jump_gbm(
    number_of_sims=4,
    trading_days=2512,
    mean_annual_return=0.05,
    mean_annual_vol=0.1,
    jumps_lamda=0.1,
    jumps_sigma=0.3,
    jumps_mu=-0.2,
    trading_days_in_year=252,
    seed=seed,
)

date = get_previous_business_day_before_today(markets="XSTO")

assets = OpenFrame(constituents=[
    OpenTimeSeries.from_df(dframe=series.to_dataframe(name="Asset", end=date, markets="XSTO"), column_nmbr=serie) for
    serie in range(series.number_of_sims)]).to_cumret()

In [15]:
simulations = 5000
points = 30

current = OpenTimeSeries.from_df(
    dframe=assets.make_portfolio(
        name="Current Portfolio",
        weight_strat="eq_weights",
    ),
)
frontier, simulated, optimum = efficient_frontier(
    eframe=assets,
    num_ports=simulations,
    seed=seed,
    frontier_points=points,
)
plotframe = prepare_plot_data(
    assets=assets,
    current=current,
    optimized=optimum,
)
simfigure, _ = sharpeplot(
    sim_frame=simulated,
    line_frame=frontier,
    point_frame=plotframe,
    point_frame_mode="markers+text",
    title=False,
    add_logo=False,
    auto_open=False,
    output_type="div",
)
simfigure = simfigure.update_layout(width=1100, height=650)

In [16]:
simfigure.show(config=figdict["config"])